In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:

iris = datasets.load_iris()

In [3]:
X = iris['data'][:,(2,3)]
X = np.c_[np.ones((X.shape[0],1)),X]
y = iris['target']
y1 = (y ==2).astype(np.int)

In [5]:
eta = 0.1
n_iterations = 1000
theta = np.random.randn(3,1)
m = X.shape[0]

def sigma(t):
    return 1 / (1 + np.exp(-t))

vsigma = np.vectorize(sigma)

for iteration in range(n_iterations):
    gradients = 1/m * X.T.dot(vsigma(X.dot(theta)) - y1.reshape(-1,1))
    theta = theta  - eta * gradients
    
def Logistic_pred(phat):
    return (phat >= 0.5).astype(np.int)

Logistic_pred(vsigma(X.dot(theta)))

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
    